In [1]:
import json
import rasterio as rio
import geopandas as gpd

from fiona.crs import from_epsg
from shapely.geometry import box
from rasterio.mask import mask

# Crop and save GeoTiff

A small notebook to crop massive sized GeoTiffs to something more manageable.

Define paths of massive image and the output path

In [2]:
data_path = "data/too_big/20181013_095233_Tri_80cm_RD_8bit_RGB_Middelburg.tif"
out_tif = "data/20181013-kortgene-highres/20181013-kortgene-allbands.tif"
tiff_file = rio.open(data_path)

Define the bounding box in WGS84 coordinates:

In [3]:
min_x, min_y = 3.774023802020978, 51.563824535249466
max_x, max_y = 3.809986860492658, 51.57916179810492
geometry = box(min_x, min_y, max_x, max_y)
geometry_crs = from_epsg(4326)
geo = gpd.GeoDataFrame({"geometry": geometry}, index=[0], crs=geometry_crs)

Open and crop the massive GeoTiff

In [4]:
with rio.open(data_path) as tiff_file:
    # Project bounding box to GeoTiff CRS
    geo = geo.to_crs(crs=tiff_file.crs.data)
    # Convert geometry to something that rasterio understands
    raster_geometry = [json.loads(geo.to_json())['features'][0]['geometry']]
    # Crop the image
    cropped_image, cropped_transform = mask(tiff_file, shapes=raster_geometry, crop=True)
    # Copy the image metadata
    cropped_meta = tiff_file.meta.copy()

Update the cropped image metadata

In [5]:
cropped_meta.update({
    "count": cropped_image.shape[0],
    "height": cropped_image.shape[1],
    "width": cropped_image.shape[2],
    "transform": cropped_transform
})

Write to output file

In [6]:
with rio.open(out_tif, "w", **cropped_meta) as dest:
    dest.write(cropped_image)